In [31]:
# %matplotlib inline

# import matplotlib.pyplot as plt
# import seaborn as sns
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
# url_1967 = "http://www.pro-football-reference.com/years/1967/draft.htm"
# html = urlopen(url_1967)
# soup = BeautifulSoup(html, "html.parser")

In [32]:
# column_headers = [th.getText() for th in 
#                   soup.findAll('tr', limit=2)[1].findAll('th')]
# column_headers.extend(["Player_NFL_Link", "Player_NCAA_Link"])
# column_headers=[x for x in column_headers if x]
# table_rows = soup.select("#drafts tr")[1:]
def extract_player_data(table_rows):
    player_data = []
    
    for row in table_rows:  # for each row do the following

        player_list = [td.get_text()[:-4] if td.get_text().endswith(" HOF") 
                       else td.get_text() for td in row.find_all(["td","th"])]
        if not player_list:
            continue
        
        links_dict = {(link.get_text()[:-4]   # exclude the last 4 characters
                       if link.get_text().endswith(" HOF")  # if they are " HOF"
                       # else get all text, set thet as the dictionary key 
                       # and set the url as the value
                       else link.get_text()) : link["href"] 
                       for link in row.find_all("a", href=True)}
        
        player_list.append(links_dict.get(player_list[3], " "))
        
        player_list.append(links_dict.get("College Stats", " "))
        
        player_data.append(player_list)
        
    return player_data
# extract the data we want
# data = extract_player_data(table_rows)
# and then store it in a DataFrame
# df = pd.DataFrame(data, columns=column_headers)

In [47]:

draft_dfs_list = []

# a list to store any errors that may come up while scraping
errors_list = []
# The url template that we pass in the draft year inro
url_template = "http://www.pro-football-reference.com/years/{year}/draft.htm"

# for each year 
for year in range(1997, 2018): 
    
    # Use try/except block to catch and inspect any urls that cause an error
    try:
        # get the draft url
        url = url_template.format(year=year)

        # get the html
        html = urlopen(url)

        # create the BeautifulSoup object
        soup = BeautifulSoup(html, "html.parser") 

        # get the column headers
        column_headers = [th.getText() for th in 
                          soup.findAll('tr', limit=2)[1].findAll('th')]
        column_headers.extend(["Player_NFL_Link", "Player_NCAA_Link"])

        # select the data from the table using the '#drafts tr' CSS selector
        table_rows = soup.select("#drafts tr")[2:] 

        # extract the player data from the table rows
        player_data = extract_player_data(table_rows)

        # create the dataframe for the current years draft
        year_df = pd.DataFrame(player_data, columns=column_headers)

        # if it is a draft from before 1994 then add a Tkl column at the 
        # 24th position
        if year < 1994:
            year_df.insert(24, "Tkl", "")

        # add the year of the draft to the dataframe
        year_df.insert(0, "Draft_Yr", year)

        # append the current dataframe to the list of dataframes
        draft_dfs_list.append(year_df)
    
    except Exception as e:
        # Store the url and the error it causes in a list
        error =[url, e] 
        # then append it to the list of errors
        errors_list.append(error)

In [43]:
len(errors_list)

0

In [44]:
# store all drafts in one DataFrame
draft_df = pd.concat(draft_dfs_list, ignore_index=True)
draft_df.drop(draft_df[draft_df.To == 'To'].index, inplace=True)
player_ids = draft_df.Player_NFL_Link.str.extract("/.*/.*/(.*)\.",expand=False)
draft_df["Player_Id"] = player_ids

In [45]:
f=[]
k=[]
t=draft_df.To.tolist()
d=draft_df.Draft_Yr.tolist()
for i in range(0,len(t)):
    
    if t[i] == '':
        f.append(0)
        k.append(0)
    else:
        s=int(t[i])-d[i]
        f.append(s+1)
        k.append(1)
draft_df['duration'] = f
draft_df['retired'] = k
draft_df.drop(['Player_NFL_Link','Player_NCAA_Link',''], axis = 1, inplace = True)

In [46]:
draft_df.to_csv('main-big-1997-2017-s.csv')